# 阅读笔记

** 作者：方跃文 **

** Email: fyuewen@gmail.com **

** 时间：始于2017年9月12日， 结束写作于 **

** 上海纽约大学 **

** 京都大学**



# 第二章 引言

** 时间： 2017年9月12日 **

尽管数据处理的目的和领域都大不相同，但是利用python数据处理时候基本都需要完成如下几个大类的任务：

1) 与外界进行数据交互

2) 准备：对数据进行清理、修整、规范化、重塑、切片切块

3) 转换：对数据集做一些数学和统计运算以产生新的数据集，e.g. 根据分组变量对一个大表进行聚合

4) 建模和计算：将数据跟统计模型、机器学习算法联系起来

5) 展示：创建交换式的或者静态的图片或者文字摘要。


## 来自bit.ly的1.usa.gov数据

ch02中的usagov_bitly_data2012-03-16-1331923249.txt是bit.ly网站收集到的每小时快照数据。文件中的格式为JavaScript Object Notation （JSON)——一种常用的web数据格式。例如如果我们只读取某个文件中的第一行，那么所看到的结果是下面这样：